# GAN implementation by tensorflow

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility関数


In [ ]:
def save_metrics(metrics, epoch=None):
    path = "metrics"
    if os.path.isdir(path) == False:
        os.mkdir(path)

    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [ ]:
# noise[[examples, 100]]から生成した画像をplot_dim(例えば4x4)で表示
def save_imgs(images, plot_dim=(5,12), size=(24,10), epoch=None):
    path = "generated_figures"
    if os.path.isdir(path) == False:
        os.mkdir(path)
    
    examples = plot_dim[0]*plot_dim[1]
    
    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((128, 128, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()


In [ ]:
def selu(x):
    #with ops.name_scope('elu') as scope:
    alpha = 1.6732632423543772848170429916717
    scale = 1.0507009873554804934193349852946
    return scale*tf.where(x>=0.0, x, alpha*tf.nn.elu(x))

# モデル

In [ ]:
class Generator:
    def __init__(self, img_size, isBN):
        self.img_size = img_size
        self.G_h_size = 64
        self.reuse = False
        self.isBN = isBN
        self.z_dim = 100

    def __call__(self, x, training=False):
        mult = self.img_size//8
        x = tf.convert_to_tensor(x)
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            with tf.variable_scope('reshape'):
                ## start
                outputs = tf.layers.dense(x, self.z_dim*2*2, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                outputs = tf.reshape(outputs, [-1, 2, 2, self.z_dim])
                outputs = tf.layers.conv2d_transpose(outputs, self.G_h_size*mult, [4,4], strides=(2,2), padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 4x4
                if self.isBN == True:
                    outputs = tf.layers.batch_normalization(outputs)
                    outputs = tf.nn.relu(outputs)
                else:
                    outputs = selu(outputs)

                ## middle [4, 8, 16, 32, 64, 128, 256, 512] 8
                while mult > 1:
                    outputs = tf.layers.conv2d_transpose(outputs, self.G_h_size*(mult//2), [4,4], strides=(2,2), padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                    if self.isBN == True:
                        outputs = tf.layers.batch_normalization(outputs)
                        outputs = tf.nn.relu(outputs)
                    else:
                        outputs = selu(outputs)
                    mult = mult//2

                ## end
                outputs = tf.layers.conv2d_transpose(outputs, 3, [4,4], strides=(2,2), padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                outputs = tf.nn.tanh(outputs)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [ ]:
class Discriminator:
    def __init__(self, img_size, isBN):
        self.reuse = False
        self.D_h_size = 64
        self.img_size = img_size
        self.isBN = isBN

    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.2):
            return tf.maximum(x, x * leak)
        outputs = tf.convert_to_tensor(inputs)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            ## start
            outputs = tf.reshape(outputs, [-1, self.img_size, self.img_size, 3])
            outputs = tf.layers.conv2d(outputs, self.D_h_size, [4, 4], strides=2, padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))

            if self.isBN == True:
                outputs = tf.layers.batch_normalization(outputs)
                outputs = leaky_relu(outputs)
            else:
                outputs = selu(outputs)

            img_size_new = self.img_size//2
            mult = 1

            ## middle
            while img_size_new > 4:
                outputs = tf.layers.conv2d(outputs, self.D_h_size*(2*mult), [4, 4], strides=2, padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                if self.isBN == True:
                    outputs = tf.layers.batch_normalization(outputs)
                    outputs = leaky_relu(outputs)
                else:
                    outputs = selu(outputs)

                mult = mult*2
                img_size_new = img_size_new//2
            
            ## end
            outputs = tf.layers.conv2d(outputs, 1, [4, 4], strides=1, padding="VALID", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
            #outputs = tf.layers.dense(outputs, 1)
            print(outputs.get_shape())
            outputs = tf.nn.sigmoid(outputs)
            
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs

In [ ]:
class GAN:
    def __init__(self):
        self.batch_size = 128
        self.img_size = 128
        self.z_dim = 100

        self.epochs = 1000000
        self.epoch_saveMetrics = 200
        self.epoch_saveSampleImg = 200
        self.epoch_saveParamter = 10000
        self.losses = {"d_loss":[], "g_loss":[]}

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 3])
        self.Z1 = tf.placeholder(tf.float32, [None, self.z_dim])
        
        self.isBN = False

        self.g = Generator(self.img_size, self.isBN)
        self.d = Discriminator(self.img_size, self.isBN)
        self.Xg = self.g(self.Z1)

    def loss(self):
        Dr = self.d(self.X_tr)
        Dg = self.d(self.Xg)

        L_d = -tf.reduce_mean(tf.log(Dr) + tf.log(1. - Dg))
        L_g = -tf.reduce_mean(tf.log(Dg))

        return L_g, L_d

    def train(self):
        # Optimizer
        d_lr = 5e-5
        g_lr =  2e-4
        #d_lr = 2e-4
        #g_lr = 2e-4
        beta1 = 0.5
        beta2 = 0.9

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=d_lr, beta1=beta1, beta2=beta2)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=g_lr, beta1=beta1, beta2=beta2)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        #%debug
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            z_test = np.random.uniform(-1, 1, size=[60, self.z_dim])

            for epoch in range(self.epochs):

                # X_mb
                def extractXimg(path, batch_size):
                    imgs = os.listdir(path)
                    rand_id = np.random.randint(0, len(imgs), size=batch_size)
                    X_mb = np.zeros((1, self.img_size, self.img_size, 3))

                    for i in range(batch_size):
                        img = Image.open(path+"/"+imgs[rand_id[i]])

                        img_np = np.asarray(img)
                        img_np_copied = np.copy(img_np[:,:,0:3])

                        for x in range(img.size[0]): 
                            for y in range(img.size[1]):
                                if img_np[x][y][3] < 255:    # check alpha
                                    img_np_copied[x][y] = (255,255,255)
                                    #img_np_copied[x][y] = (0, 0, 0)

                        if (i==0)and (epoch % 10==1):
                            %matplotlib inline
                            plt.imshow(img_np_copied)
 
                        X_mb = np.vstack((X_mb, img_np_copied[np.newaxis, :]))
                        img.close()
                    X_mb = X_mb[1:,:]
                    return X_mb
                
                for _ in range(1):
                    # 訓練データを抜粋
                    z1 = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim])

                    X_mb = extractXimg(str(self.img_size) + "x" + str(self.img_size) + "x3resized_faceimgs", self.batch_size).astype(np.float32)

                    X_mb = X_mb/255
                    
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={self.X_tr: X_mb, self.Z1:z1})

                # train G
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={self.X_tr: X_mb, self.Z1: z1})

                # 結果をappend
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                if epoch % 100 == 0:
                    print("epoch:" + str(epoch))

                # lossの可視化
                if epoch % self.epoch_saveMetrics == 1:
                    save_metrics(self.losses, epoch)

                # 画像の変換テスト
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.Z1: z_test})
                    save_imgs(img, epoch=str(epoch))

                # parameterのsave
                if epoch % self.epoch_saveParamter == 1:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/acgan_model" + str(epoch) + ".ckpt")
       

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
gan = GAN()
gan.train()